In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
%run env.py
%run rl-helpers.py

In [3]:
from agents.dqn import DQNAgent, ConvQNetworkFactory
from agents.random import RandomAgent

In [4]:
env = WindowedGridView(DeliveryDrones(), radius=3)
env.env_params.update({'n_drones': 5, 'pickup_reward': 1, 'discharge': 2, 'rgb_render_rescale': 2.0})

In [5]:
"""DQN with conv. Q-network"""
dqn_agent_1 = DQNAgent(
    env, ConvQNetworkFactory(env, conv_layers=[
        {'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': 1},
        {'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': 1},
        {'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': 1},
        {'out_channels': 64, 'kernel_size': 3, 'stride': 1, 'padding': 1},
        {'out_channels': 64, 'kernel_size': 3, 'stride': 1, 'padding': 1},
        {'out_channels': 64, 'kernel_size': 3, 'stride': 1, 'padding': 1}
    ], dense_layers=[256]),
    gamma=0.95, epsilon_start=1, epsilon_decay=0.99, epsilon_end=0.01, memory_size=10000, batch_size=64, target_update_interval=5)

dqn_agent_2 = DQNAgent(
    env, ConvQNetworkFactory(env, conv_layers=[
        {'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': 1},
        {'out_channels': 64, 'kernel_size': 3, 'stride': 1, 'padding': 1}
    ], dense_layers=[256]),
    gamma=0.95, epsilon_start=1, epsilon_decay=0.99, epsilon_end=0.01, memory_size=10000, batch_size=64, target_update_interval=5)

In [6]:
from agents.curiosity import CuriosityDQNAgent

dqn_factory = ConvQNetworkFactory(env, conv_layers=[
        {'out_channels': 32, 'kernel_size': 3, 'stride': 2, 'padding': 1},
        {'out_channels': 32, 'kernel_size': 3, 'stride': 2, 'padding': 1},
        {'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': 1},
        {'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': 1},
    ], dense_layers=[256])

agent_3 = CuriosityDQNAgent(env, dqn_factory, gamma=0.98, epsilon_start=1, epsilon_decay=0.995, epsilon_end=0.01, memory_size=50000,
                                  batch_size=32, target_update_interval=-1)

In [7]:
# Reset environment with those parameters
env.reset()

# Setup random opponents
agents = {drone.index: RandomAgent(env) for drone in env.drones}
agents[0] = dqn_agent_1
agents[1] = dqn_agent_2
agents[2] = agent_3

# Create trainer
trainer = MultiAgentTrainer(env, agents, reset_agents=True, seed=0)

In [8]:
for agent in agents.values():
    agent.is_greedy = False

trainer.train(500)
plot_rolling_rewards(trainer.rewards_log)

Training agents:   5%|▍         | 23/500 [00:00<00:02, 172.80it/s]


TypeError: add(): argument 'other' (position 1) must be Tensor, not tuple

In [ ]:
for agent in agents.values():
    agent.is_greedy = True

rewards_log = test_agents(env, agents, n_steps=1000)
plot_cumulative_rewards(rewards_log)